# prototypes 2

In [7]:
#!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 KB 8.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 KB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 34.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 KB 19.2 MB/s eta 0:00:00
     

In [9]:
#!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 62.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 KB 37.9 MB/s eta 0:00:00


In [7]:
#!pip install --upgrade pillow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 47.7 MB/s eta 0:00:0000:0100:01


In [13]:
import random
from PIL import Image
import PIL.Image
import PIL.ExifTags

PIL.Image.ExifTags = PIL.ExifTags

from datasets import load_dataset
from transformers import AutoTokenizer

ds = load_dataset("deepcopy/MathWriting-Human", split="train+val+test")

train_set = ds.filter(lambda x: x["split_tag"] == "train")
val_set = ds.filter(lambda x: x["split_tag"] == "val")
test_set = ds.filter(lambda x: x["split_tag"] == "test")

def resize_image(example, size=(224, 224)):
    img = example["image"]
    img = img.convert("RGB")
    img = img.resize(size)
    example["image"] = img
    return example

train_set = train_set.map(resize_image, num_proc=12)
val_set = val_set.map(resize_image, num_proc=12)
test_set = test_set.map(resize_image, num_proc=12)

latex_pool = train_set["latex"]

def add_binary_label(example, latex_list):
    if random.random() > 0.5:
        example["label"] = 1
        example["latex_used"] = example["latex"]
    else:
        wrong_latex = random.choice(latex_list)
        while wrong_latex == example["latex"]:
            wrong_latex = random.choice(latex_list)
        example["label"] = 0
        example["latex_used"] = wrong_latex
    return example

train_set = train_set.map(lambda x: add_binary_label(x, latex_pool), num_proc=12)
val_set = val_set.map(lambda x: add_binary_label(x, latex_pool), num_proc=12)
test_set = test_set.map(lambda x: add_binary_label(x, latex_pool), num_proc=12)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_latex(example):
    tokens = tokenizer(example["latex_used"], truncation=True, padding="max_length", max_length=64)
    example["latex_ids"] = tokens["input_ids"]
    return example

train_set = train_set.map(tokenize_latex, num_proc=12)
val_set = val_set.map(tokenize_latex, num_proc=12)
test_set = test_set.map(tokenize_latex, num_proc=12)

tf_train = train_set.to_tf_dataset(
    columns=["image", "latex_ids"],
    label_cols=["label"],
    shuffle=True,
    batch_size=256
)

tf_val = val_set.to_tf_dataset(
    columns=["image", "latex_ids"],
    label_cols=["label"],
    shuffle=False,
    batch_size=256
)

tf_test = test_set.to_tf_dataset(
    columns=["image", "latex_ids"],
    label_cols=["label"],
    shuffle=False,
    batch_size=256
)

Filter:   0%|          | 0/253182 [00:00<?, ? examples/s]


AttributeError: module 'PIL.ExifTags' has no attribute 'Base'

In [3]:
from tensorflow import keras
from tensorflow.keras import layers


img_input = keras.Input(shape=(224, 224, 3), name="image", dtype="float32")
x_img = layers.Rescaling(1.0 / 255)(img_input)
x_img = layers.Conv2D(32, 3, activation="relu")(x_img)
x_img = layers.MaxPooling2D()(x_img)
x_img = layers.Conv2D(64, 3, activation="relu")(x_img)
x_img = layers.GlobalAveragePooling2D()(x_img)

txt_input = keras.Input(shape=(64,), name="latex_ids", dtype="int32")
x_txt = layers.Embedding(input_dim=50257, output_dim=32)(txt_input)
x_txt = layers.GlobalAveragePooling1D()(x_txt)

x = layers.Concatenate()([x_img, x_txt])
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=[img_input, txt_input], outputs=output, name="version-cv")
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()
model.fit(tf_train, epochs=3)

Model: "version-cv"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ image[0][0]       │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 222, 222,  │        896 │ rescaling_2[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 111, 111,  │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latex_ids           │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d_2[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 64, 32)    │  1,608,224 │ latex_ids[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ conv2d_5[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ embedding_2[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 96)        │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     12,416 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │        129 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,640,161 (6.26 MB)

 Trainable params: 1,640,161 (6.26 MB)

 Non-trainable params: 0 (0.00 B)

NameError: name 'tf_train' is not defined

In [ ]:
model.save("version-cv.keras")

In [ ]:
from tensorflow import keras

# Load model
model = keras.models.load_model("my_model.keras")

# HDF5
model = keras.models.load_model("my_model.h5")